# RLHF
created by cyj
## 1、PG
### 1、PG的优化目标
我们希望学习一个参数化的策略$\pi_\theta(a \mid s)$，使得期望回报最大化：
$$
J(\theta)
=
\mathbb{E}_{\tau \sim \pi_\theta}
\left[ R(\tau) \right]
$$

- $\theta$ 表示策略网络参数

- $\tau = (s_0, a_0, s_1, a_1, \ldots)$ 表示由策略 $\pi_\theta$ 采样得到的轨迹

- $R(\tau)$ 表示轨迹的累计回报。  
策略梯度不是在“学对的动作”，而是在“提高产生好结果的概率”  
### 2、PG的核心公式
$$
\nabla_\theta J(\theta)
=
\mathbb{E}_{\tau \sim \pi_\theta}
\left[
\sum_{t}
\nabla_\theta \log \pi_\theta(a_t \mid s_t)\; G_t
\right]
$$
- $G_t = \sum_{t'=t}^{T} r_{t'}$ 表示从时间步 $t$ 开始的累计回报（return）。  
- $\log \pi_\theta(a_t \mid s_t)$ 是已采样动作的对数概率   
该公式将不可微的环境奖励问题，转化为对 $\log \pi_\theta$ 的可微优化。



### 3、Return的定义（Monte Carlo）
在最基本的REINFRORCE算法中，Return定义为：
$$
G_t
=
\sum_{k=t}^{T}
\gamma^{k-t} r_k
$$
- $r_k$ 是时间步 $k$ 的即时奖励
- $\gamma \in [0, 1]$ 是折扣因子，控制未来奖励的重要性
这种估计方式是无偏但是方差高的：  
Monte Carlo return 是无偏估计，因为它完整采样了真实轨迹的所有未来奖励；
但它是高方差估计，因为同一状态下的未来轨迹具有巨大的随机性

在 Policy Gradient 中$G_t$不是单步 reward，而是“从时间步 𝑡开始的累计回报（return）”；它由 reward 组成，因此可以理解为一种“综合 reward 分数”，但不是原始 reward 本身

### 4、PG中的Loss形式
由于实际训练中我们通常最小化loss，可以将目标函数取负，得到等价的loss形式：
$$
L^{PG}(\theta)
=
-
\mathbb{E}_{\tau \sim \pi_\theta}
\left[
\sum_{t}
\log \pi_\theta(a_t \mid s_t)\; G_t
\right]
$$

这里可以理解为，以回报$G_t$为权重，对已采样动作的 log-likelihood 进行最大化,让模型以后更容易生成这些动作


### 5、Baseline 与 方差降低
对于任意不依赖于动作$a_t$ 的函数 $b(s_t)$，有：
$$
\mathbb{E}_{a_t \sim \pi_\theta}
\left[\nabla_\theta \log \pi_\theta(a_t \mid s_t)\; b(s_t) 
\right]
=
b(s_t)\; \mathbb{E}_{a_t \sim \pi_\theta}
\left[\nabla_\theta \log \pi_\theta(a_t \mid s_t)
\right]
= 
0
$$
因此，可以将 $b(s_t)$ 从梯度中减去，而不引入偏差：
$$
\nabla_\theta J(\theta)
=
\mathbb{E}_{\tau \sim \pi_\theta}
\left[
\sum_{t}
\nabla_\theta \log \pi_\theta(a_t \mid s_t)\; (G_t - b(s_t))
\right]
$$
- $b(s_t)$ 称为 baseline，常用的选择是状态值函数 $V^\pi(s_t)$  
- 通过减去 baseline，可以显著降低梯度的方差，提高训练稳定性  
- 直观上，baseline 起到了“中心化”回报的作用，使得只有当 $G_t$ 高于 baseline 时，才会增加该动作的概率

### 6、Advantage Function
当baseline为状态价值函数$b(s_t) = V^\pi(s_t)$时，定义优势函数（Advantage Function）为：
$$
A^\pi(s_t, a_t)
=
G_t - V^\pi(s_t)
$$
此时，Policy Gradient公式可写为：
$$
\nabla_\theta J(\theta)
=
\mathbb{E}_{\tau \sim \pi_\theta}
\left[
\sum_{t}
\nabla_\theta \log \pi_\theta(a_t \mid s_t)\; A^\pi(s_t, a_t)
\right]
$$
Value function 预测如果我从这里继续走，大概能拿多少 reward

### 7、状态价值函数如何计算
状态价值函数的目的是降低方差， $V^\pi(s_t)$ 可以通过多种方法估计，常见的方法包括：
- **蒙特卡洛估计**：使用多次采样的完整轨迹来计算平均回报（最直观的方法，但是最少用）
- **时序差分学习（TD Learning）**：使用单步奖励和下一个状态的估计值来更新当前状态的价值估计
- **GAE（Generalized Advantage Estimation）**：结合多步回报和TD误差，平衡偏差和方差（PPO/A2C的选择）
- **函数逼近**：使用神经网络等函数逼近器来学习状态价值函数，通过最小化预测值与实际回报之间的误差来训练网络（最常见）

GAE的核心公式是：
$$
A^{GAE(\gamma, \lambda)}_t
=
\sum_{l=0}^{\infty}
(\gamma \lambda)^l \delta_{t+l}
$$
其中，$\delta_t$ 是时序差分误差，定义为：
$$
\delta_t
=
r_t + \gamma V(s_{t+1}) - V(s_t)
$$
- $\lambda \in [0, 1]$ 是一个调节参数，控制偏差与方差的权衡
- r_t 是时间步 t 的即时奖励
- $V(s_t)$ 是状态 $s_t$ 的价值估计
- 当 $\lambda = 0$ 时，GAE 退化为单步 TD 误差
- 当 $\lambda = 1$ 时，GAE 退化为蒙特卡洛估计
直觉含义就是我这一步之后之后，比我原来预期的更好还是更差



所以我们可以总结一下完整的使用GAE作为优势函数的Policy Gradient算法步骤：  
Step1:Value Network前向，计算每个时间步的状态价值估计 $V(s_t)$  
Step2:计算时序差分误差 $\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)$，其中 $r_t$ 是时间步 $t$ 的即时奖励，r_t是通过环境反馈得到的奖励
Step3:计算GAE优势估计：
$$
A^{GAE(\gamma, \lambda)}_t
=
\sum_{l=0}^{\infty}
(\gamma \lambda)^l \delta_{t+l}
$$ 
Step4:使用GAE优势函数更新策略网络参数：
$$
\nabla_\theta J(\theta)
=
\mathbb{E}_{\tau \sim \pi_\theta}
\left[
\sum_{t}
\nabla_\theta \log \pi_\theta(a_t \mid s_t)\; A^{GAE(\gamma, \lambda)}_t
\right]
$$
Steap5:Value Network的更新，最小化以下损失函数：
$$
L^{V}(\phi)
=
\mathbb{E}_{\tau \sim \pi_\theta}
\left[
\sum_{t}
\left(V_\phi(s_t) - G_t\right)^2
\right]
$$
- 其中，$V_\phi(s_t)$ 是参数为 $\phi$ 的价值网络的估计值，$G_t$ 是从时间步 $t$ 开始的累计回报


### 8、PG伪代码
不用GAE，只用Monte Carlo Return，有policy，有value，不用PPO的ratio/clip
公式就是：
$$
\nabla_\theta J(\theta)
=
\mathbb{E}_{\tau \sim \pi_\theta}
\left[
\sum_{t}
\nabla_\theta \log \pi_\theta(a_t \mid s_t)\; (G_t - V_\phi(s_t))
\right]
$$

In [ ]:
'''
Initialize policy network πθ(a | s)
Initialize value network Vφ(s)

for iteration = 1, 2, ... do:
    Collect trajectories using current policy πθ

    for each trajectory:
        for each timestep t:
            compute return:
                G_t = sum_{k=t}^T γ^{k-t} r_k
            compute advantage:
                A_t = G_t - Vφ(s_t)

    Policy update:
        L_policy = - E[ log πθ(a_t | s_t) * A_t ]
        update θ by gradient descent

    Value update:
        L_value = E[ (Vφ(s_t) - G_t)^2 ]
        update φ by gradient descent
'''



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Policy Model
class PolicyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.logits = nn.Parameter(torch.zeros(2))  # 初始化两个动作的logits
        
    def forward(self):
        return F.softmax(self.logits, dim=-1)  # dim=-1在softmax中表示对每一行的类别概率进行计算

# 价值网络
# class ValueNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.value = nn.Parameter(torch.zeros(1))  # 价值函数，只有一个数值
        
#     def forward(self):
#         return self.value
class ValueNet(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=16):
        super().__init__()
        # 定义一个两层的简单神经网络
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # 第一层
        self.fc2 = nn.Linear(hidden_dim, 1)  # 输出层，输出一个标量（状态的价值估计）

    def forward(self, x):
        x = F.relu(self.fc1(x))  # 使用ReLU激活函数
        return self.fc2(x)  # 输出最终的价值


# 定义环境
class SimpleEnv:
    def __init__(self):
        self.action = [0, 1]  # 两个动作，0和1
        self.reward = [1, -1]  # 每个动作的奖励
    
    def reset(self):
        return 0
    
    def step(self, action):
        # 根据动作返回奖励，这里直接返回对应的奖励
        reward = self.reward[action]
        return reward

def train(env, policy_net, value_net, optimizer, num_episodes=1000, gamma=0.99):
    total_rewards = []  # 存储每个回合的总奖励
    logits_history = []  # 存储logits的变化

    for episode in range(num_episodes):
        state = env.reset()
        state = torch.tensor([state], dtype=torch.float32)  # 转换为tensor
        done = False
        episode_reward = 0
        
        while not done:
            # 获取策略网络的输出
            action_probs = policy_net()
            dist = torch.distributions.Categorical(action_probs)  # 计算动作分布
            action = dist.sample()  # 从计算的动作分布中采样一个动作
            log_prob = dist.log_prob(action)  # 计算该动作的对数概率
            
            reward = env.step(action.item())  # 执行动作并获得奖励
            value = value_net(state)  # 现在使用状态输入来计算当前的价值估计
            
            td_target = reward + gamma * value
            td_error = td_target - value
            
            # 计算策略损失，从计算图中分离TD误差，防止影响到actor网络
            actor_loss = -log_prob * td_error.detach() 
            
            # 计算value损失，TD误差的平方
            critic_loss = td_error.pow(2)
            
            # 总损失
            loss = actor_loss + critic_loss  # 总损失 = 策略损失 + 价值损失
            
            # 反向传播
            optimizer.zero_grad()  # 清除旧的梯度
            loss.backward()  # 计算新的梯度
            
            # 更新网络参数
            optimizer.step()
            
            episode_reward += reward
            done = True  # 每个回合结束之后停止训练

        total_rewards.append(episode_reward)
        logits_history.append(policy_net.logits.detach().numpy())  # 记录logits的变化

    return total_rewards, logits_history


# 5. 初始化环境、模型和优化器
env = SimpleEnv()  # 初始化环境
policy_net = PolicyNet()  # 初始化策略网络（Actor）
value_net = ValueNet()  # 初始化价值网络（Critic）
optimizer = optim.Adam(list(policy_net.parameters()) + list(value_net.parameters()), lr=0.01)  # 使用Adam优化器

# 6. 开始训练并记录结果
total_rewards, logits_history = train(env, policy_net, value_net, optimizer)

# 7. 可视化训练过程
# 绘制每个回合的总奖励
plt.figure(figsize=(10, 6))
plt.plot(total_rewards)
plt.title('Total Rewards over Episodes')
plt.xlabel('Episodes')
plt.ylabel('Total Reward')
plt.show()

# 绘制策略网络的logits变化
logits_history = torch.tensor(logits_history)
plt.figure(figsize=(10, 6))
plt.plot(logits_history[:, 0], label='Logits for Action 0')
plt.plot(logits_history[:, 1], label='Logits for Action 1')
plt.title('Logits for Actions over Episodes')
plt.xlabel('Episodes')
plt.ylabel('Logits')
plt.legend()
plt.show()


# 2、PPO
PPO的核心思想是通过限制策略更新的幅度，来稳定训练过程，防止策略发生剧烈变化。  
PPO的目标函数可以表示为：
$$
L^{PPO}(\theta)
=
\mathbb{E}_{\tau \sim \pi_{\theta_{old}}}
\left[
\min \left(
r_t(\theta) A_t,
\text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon) A_t
\right)
\right]
$$
- $r_t(\theta) = \frac{\pi_\theta(a_t \mid s_t)}{\pi_{\theta_{old}}(a_t \mid s_t)}$ 是新旧策略的概率比率
- $A_t$ 是优势函数，通常使用GAE估计
- $\epsilon$ 是一个小的超参数，控制裁剪范围，通常取值为0.1或0.2

PPO另外的核心改进的是Minibatch和多轮更新：
- 在每次策略更新时，PPO会从采样的轨迹中随机抽取多个小批量（minibatch）进行多轮优化
- 这种方式提高了数据利用率，减少了方差，使得训练更加稳定和高效



# 3、GRPO
**motivation**：
- 从PG到PPO，都需要训练一个与policy model大小相当大critic model来估计value function，这在大模型时代是非常不现实的。
- 此外，在LLM推理任务中，reward通常只会在生成序列的末尾给出，例如，最终的答案是不是正确，这使得为每一个token训练一个精准的value function变得更加困难。  

**核心创新**：
- 放弃训练一个独立的critic model，转而直接使用reward signal来计算优势函数。具体来说，对于每一个prompt，policy model会使用旧的策略采样生成一个包含G个不同回答的group，通过比较这个组各个回答的奖励，来估计每个回答的advantage。  

GRPO的目标函数可以表示为：
$$
L^{GRPO}(\theta)
=
\mathbb{E}_{\tau \sim \pi_{\theta_{old}}}
\left[
\min \left(
r_t(\theta) A^{GR}_t,
\text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon) A^{GR}_t
\right)
\right]
$$
- 其中，$A^{GR}_t$ 是基于group reward计算的优势函数，具体计算方式如下：
$$
A^{GR}_i
=(r_i - mean({{r_j}^G_{j=1}}))/std({r_j}^G_{j=1})
$$
- $R_i$ 是第i个回答的reward，$G$是group中回答的数量

GRPO中，reward 是 针对整个序列给出的，advantage也是针对整个序列给出的
但是在计算梯度时，仍然是对每个时间步（token）的 log-likelihood 进行加权求和，这是为了确保Advantage这个全局奖励能平摊到每一帧动作上（Token），不会因为动作时间长就导致梯度爆炸：
$$
\nabla_\theta J(\theta)
=
\mathbb{E}_{\tau \sim \pi_\theta}
\left[
\sum_{t}    
\nabla_\theta \log \pi_\theta(a_t \mid s_t)\; A_t
\right]

原论文中为了训练稳定性，使用了一个无偏 KL 估计器（通常被称为 K3 估计器的一个变种）


# 4 、Dr.GRPO  
1、回答级别的长度偏差，源于目标函数中的1/oi像，当Advantage为正时，这个因子会给予较短的正确回答更大的梯度更新，鼓励模型生成简短的错误答案，当优势为负的时候，他使较长的错误回答受到更小的惩罚，从而鼓励模型胡言乱语，生成更长的错误回答。  
2、问题级别的难度偏差，源于目标函数中的1/std，对于那些所有回答都正确（非常简单）或都错误（非常困难）的问题，组内奖励的标准差会非常小，接近于零。这会导致优势值被极大地放大，使得这些简单或困难的问题在梯度更新中占据过大的权重，从而产生对特定难度的偏好。  

解决方案: Dr. GRPO ("Doctor" GRPO) 的解决方案非常直接：移除这两个导致偏差的归一化项。即，在计算策略梯度时不除以回答长度 ，在计算优势时不除以奖励的标准差。修正后的无偏策略梯度更接近于标准的PPO目标，其优势项简化为中心化的回报  

它放弃了优势标准化带来的梯度稳定性，以换取对所有难度问题一视同仁的无偏学习信号。  

虽然方差归一化在通用RL中是一个有用的工具，但在特定应用场景下，它可能引入不期望的归纳偏置。  
后续算法（如LitePPO）则采取了折衷方案，在更大的批次（batch）级别上计算方差，以获得更鲁棒的估计  

# 5、GSPO
在对大规模模型（特别是MoE架构）进行GRPO训练时，观察到了“灾难性且不可逆的模型崩溃”现象，既然奖励是授予整个序列的，那么重要性采样和截断也应该在序列（sequence）级别进行。GSPO定义了序列级别的重要性比率 ，并对其进行长度归一化以减小方差，其目标函数相应地变为在序列级别进行截断

# 6、GMPO
问题诊断: GRPO的目标函数本质上是在优化token级别重要性加权奖励的算术平均值。算术平均值的一个众所周知的弱点是它对异常值（outliers）非常敏感。在RL训练中，某些token可能会因为策略的剧烈波动而产生极端的重要性采样比率，这些异常值会主导整个梯度更新，导致策略更新不稳定。
GMPO (Geometric-Mean Policy Optimization) 提出用几何平均值来替代算术平均值。几何平均值在数学上对极端大值不那么敏感


# 7、**DAPO**
要解决几个在规模化训练的中常见的棘手问题：
- 熵崩溃：在RL初始阶段，策略的熵（随机性）可能会迅速下降，导致模型过早地变得确定化，从而限制了探索，陷入局部最优
- 奖励噪声：对于因超出长度限制而被截断的回答，简单的奖励设计可能会惩罚一个本应正确的推理过程，从而引入噪声，影响训练稳定性
- 梯度消失：一个batch中所有的回答都正确或者错误，ad=0，这样的batch对于policy的update没有任何贡献

解决方法：  
1、Clip-Higher:截断区间的上下界限解耦，设置一个比下界更高的上界，允许正向更新更大幅度地增加好回答的概率，从而增强了策略的熵，促进了生成样本的多样性和探索。  
2、Dynamic Sampling:数据过滤，在训练过程中，会主动采样并且丢弃那些所有回答都正确或者错误的组，，确保每个用于梯度计算batch都包含了正确和错误的样本。根据当前策略的性能动态调整采样的回答数量G。在训练初期，使用较大的G以增加多样性和探索；随着训练进展，逐渐减小G以集中计算资源在更有价值的样本上，从而提高训练效率和稳定性。  
3、token-level Policy：原始的GRPO损失在序列级别进行平均，这会导致长序列中的每个token对总损失的贡献被稀释。分母改成批次内所有token的总数
4、Overlong Reward Shaping：在训练中屏蔽掉被截样本的损失计算，从而避免了由于奖励噪声引起的训练不稳定性。
5、soft Overlong Punishment:对于被截断的回答，不是简单地将奖励设为0，而是根据回答的实际长度给予一个递减的奖励。这种方式能够更细粒度地反映回答的质量，减少了由于截断引入的噪声对训练的影响。


# 8、**DPO**
DPO (Direct Preference Optimization) 提出了一种全新的方法，通过直接优化偏好数据来训练策略模型，而不依赖于传统的强化学习框架。DPO的核心思想是将偏好数据转化为一个对数几率比（log-odds ratio），并通过最大化这个比率来优化策略。
DPO的目标函数可以表示为：
$$
L^{DPO}(\theta)
=
- \mathbb{E}_{(x, y^+, y^-) \sim D}
\left[
\log \sigma \left(\frac{1}{\beta} \left( \log \pi_\theta(y^+ \mid x) - \log \pi_\theta(y^- \mid x) \right)\right)
\right]
$$
- 其中，$(x, y^+, y^-)$ 表示一个偏好对
- $\sigma$ 是sigmoid函数
- $\beta$ 是一个温度参数，控制偏好强度

sigmoid函数将对数几率比映射到0到1之间，表示模型对偏好对的预测概率。通过最大化这个概率，DPO直接优化模型以符合人类的偏好，而不需要显式地建模奖励函数或价值函数。